In [1]:
# Run this to download necessary libraries (If you have done this in the past in any notebook, you don't need to do this)
!pip install transformers[sentencepiece,torch] # HG Transformer’s Library
!pip install datasets # HG Dataset’s Library
!pip install huggingface_hub # HG sharing Library
!pip install torch # Pytorch
# Remember to turn on T4 GPU Accelerator under 'Runtime'->'Change runtime type'
import torch

if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("Using CPU")

# ***Download These Libraries As Well***
!pip install numpy
!pip install evaluate
!pip install accelerate -U

  Using cached accelerate-0.29.3-py3-none-any.whl (297 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached datasets-2.19.0-py3-none-any.whl (542 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
Using CPU
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [2]:
# James's 3.2 Preprocessing/Tokenization for Trainer API process
# Import Datasets, AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

# Define the dataset and Tokenizer
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Create Function to Map
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True, padding = False, return_token_type_ids=True)

# Call the .map function
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# Define DataCollatorWithPadding for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [3]:
# James's 3.3 Using Trainer API
# Define Model with specific head for its designed task

# Import AutoModel
from transformers import AutoModelForSequenceClassification
# num_labels=2 means the 2 catergories: Whether a pair of paraphrases are equivalent or not
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# OPTIONAL Keep Track of Progress

# Import Neccessary Libraries
from transformers import TrainingArguments # For intializing TrainingArguments
import numpy as np # For the argmax
# Defing Training Arguments
training_args = TrainingArguments(
    output_dir='/content/results',			# Output directory
    num_train_epochs=3,                 # Number of passes it does through the data
    per_device_train_batch_size=16,     # How many batches each device (GPU's) should handle (Google Colab has 2 GPU's)
    learning_rate=5e-5,                 # How many small steps it does for each calculation (The smaller the better)
    evaluation_strategy='steps',        # When the trainer evaluates the model either after 'steps' or 'epoch'
)

# Using evaluate.load (HG course's way)
import evaluate
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# # Using load_metric (Alternative Method without downloading new package, same results)
# from datasets import load_metric
# def compute_metrics(eval_preds):
#     metric = load_metric("glue", "mrpc")
#     logits, labels = eval_preds
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

In [ ]:
# Training the Model
# Import Trainer API Library
from transformers import Trainer

# Put Previous things into Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Call the Trainer to actually train
# If this takes more than 5 minutes on google colab, check your runtime type
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 